<img style="float: left;" src="D2Klogo.png" width="20%"> 

<center>

# Welcome to COMP 680 #
### Statistics for Computing and Data Science ###

### Week 11 Linear Regression ###

# Statistical Modeling in Python

Modeling plays a significant role in data analysis and builds upon fundamental concepts of statistics.  By fitting models to data we are able to accomplish the following:

* __Estimate__ distributional properties of variables, potentially conditional on other variables.

* Concisely __summarize relationships__ between variables, and make inferential statements about those relationships.

* __Predict__ values of variables of interest conditional on values of other predictor varialbes, and characterize prediction uncertainty.

We will be using Python libaray [statsmodels](https://www.statsmodels.org/stable/index.html). What's nice about this library is that all the models follow the similar structure and syntax. For example:

* Ordinaray Lease Square Regression [OLS](https://www.statsmodels.org/stable/generated/statsmodels.regression.linear_model.OLS.html)
* Generalized Linear Model [GLM](https://www.statsmodels.org/stable/glm.html)
* Generalized Additive Models [GAM](https://www.statsmodels.org/stable/gam.html)
* Linear Mixed Effects Models [MIXEDLM](https://www.statsmodels.org/stable/mixed_linear.html)

Feel free to read up on these sub-libraries and their use cases. In next few weeks we will see examples of these models.

In [ ]:
### standard imports
import numpy as np
import pandas as pd
import statsmodels.api as sm

%matplotlib inline
import matplotlib.pyplot as plt
plt.rcParams["patch.force_edgecolor"] = True
plt.rcParams["figure.figsize"] = (8,6)
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")

## Simple linear regression



In [ ]:
galton = pd.read_csv('galton.csv')
galton

In [ ]:
# scatter plot to visalize linear association
galton.plot(kind='scatter', x='midparentHeight', y='childHeight')
plt.title('Parents height vs. child height');

Simple linear regression model:
$$y_i = \beta_0 + \beta_1 x_i + \epsilon_i$$

We are going to use 3 ways to fit this simple model:
- calculate slope and intercept using formulas:
$$\hat{\beta_1} = \frac{\sum_{i=1}^n(x_i - \bar{x}_n)(y_i - \bar{y}_n)}{\sum_{i=1}^n(x_i - \bar{x}_n)^2} =r_{XY} \frac{s_Y}{s_X}, \quad \hat{\beta_0} = \bar{y}_n - \hat{\beta_1}\bar{x}_n $$
- use scipy optimization function to [minimize](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.minimize.html) residual sum of squares (equivalent to maximize Gaussian likelihood)
- use statsmodels build-in ordinary least square regression function [OLS](https://www.statsmodels.org/stable/generated/statsmodels.regression.linear_model.OLS.html)

In [ ]:
def estimate_coef(x, y):
    """A function to estimate simple linear regression coefficients:
    input x: independent variable 
    input y: dependent variable
    """
    # number of observations/points
    # n = np.size(x)
    # mean of x and y vector
    mu_x = np.mean(x)
    mu_y = np.mean(y)
    # calculate simple linear regression coefficients using formula
    SS_xy = np.sum((x-mu_x)*(y-mu_y))
    SS_xx = np.sum((x-mu_x)**2)
    # slope b_1
    b_1 = SS_xy / SS_xx
    # intercept b_0
    b_0 = mu_y - b_1*mu_x
 
    return(b_0, b_1)

In [ ]:
b = estimate_coef(x=galton['midparentHeight'], y=galton['childHeight'])
b

In [ ]:
def RSS(b, x, y):
    """A function to calculate the residual sum of squares:
    input b: regression coefficients
    input x: independent variable 
    input y: dependent variable
    """
    RSS = np.sum((y - (b[0] + b[1]*x))**2)
    return(RSS)

In [ ]:
# call RSS function on calculated OLS intercept and slope
RSS(b=b, x=galton['midparentHeight'], y=galton['childHeight'])

In [ ]:
# call RSS function and apply random intercept and slope
# what do you expect RSS to compare to the one above?
RSS(b=[0, 1], x=galton['midparentHeight'], y=galton['childHeight'])

In [ ]:
# scipy has a built-in minimize function
from scipy.optimize import minimize
# need to input: function, random initial value to start opt algorithm, arguments for the function
res = minimize(fun=RSS, x0=[0,1], args=(galton['midparentHeight'], galton['childHeight']))
# return argmin 
res.x

In [ ]:
# make prediction and add to df 
galton['childPredictedHeight'] = b[0] + b[1]*galton['midparentHeight']
galton

In [ ]:
# plot original data: x vs. y
plt.scatter(x=galton['midparentHeight'], y=galton['childHeight'], color='blue', alpha=0.4, label='data')
# plot model prediction on x values: x vs. y_hat
plt.scatter(x=galton['midparentHeight'], y=galton['childPredictedHeight'], color='orange', alpha=0.5, zorder=3, label='y_hat')
# plot the OLS line
plt.plot(galton['midparentHeight'], galton['childPredictedHeight'], color='red', linewidth=2, label='OLS line')
plt.xlabel('midparentHeight')
plt.ylabel('childHeight')
plt.legend();

In [ ]:
# calculate residual and add to df
galton['residual'] = galton['childHeight'] - galton['childPredictedHeight']
galton

In [ ]:
# original scatter plot of data
galton.plot(kind='scatter', y='childHeight', x='childPredictedHeight')
# residual plot: diagnostic plots for liear regression
galton.plot(kind='scatter', y='residual', x='childPredictedHeight');

In [ ]:
### OLS Model
# from_formula can create a model from a formula and dataframe
# otherwise need to manually create design matrix
model = sm.OLS.from_formula('childHeight ~ midparentHeight', data=galton)
result = model.fit()
# print model summary
result.summary()
# to print out parameters only use
#result.params

Notice there are a lot of extra information in the summary:
 - [Omnibus test](https://en.wikipedia.org/wiki/Omnibus_test): general test for overall model fit, test whether the explained variance in a set of data is significantly greater than the unexplained variance.
 - [Durbin-Watson test](https://en.wikipedia.org/wiki/Durbin%E2%80%93Watson_statistic): test for autocorrelation, between 1 and 2 should be Ok.  
 - [Jarque-Bera (JB) test](https://en.wikipedia.org/wiki/Jarque%E2%80%93Bera_test): test for normality using Skewness and Kurtosis:
     - Skew: should be 0 for perfect normal distribution.
     - Kurtosis: should be 3 for perfect normal distribution.
 - Condition number: test for multicollinearity, not applicable for simple linear regression. Rule of thumb should be smaller than 30.

In [ ]:
# does the response look normal?
galton['childHeight'].plot(kind='hist', bins=30);

How to test if the normal assumption is approperiate?
- plot the y distribution
- plot the residual plot
    - scatter plot: residual vs. predicted value
    - Q-Q plot: residual quantile vs. normal quantile 
- hypothesis testing:
    - [scipy.stats.normaltest](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.normaltest.html)

In [ ]:
# to get residual values from model results: result.resid
sm.qqplot(result.resid, line ='45');

In [ ]:
import scipy.stats as st

norm_test = st.normaltest(galton['residual'])
norm_test

### R-squared and correlation

The primary summary statistic for assessing the strength of a predictive relationship in a regression model is the $R^2$, which is shown to be 0.103 in the regression output above. This means that $10\%$ of the variation in child height is explained by parents' heights.  Note that this value is exactly the same as the squared Pearson correlation coefficient between the independent variable x and dependent variable y, as shown below:

In [ ]:
cc = np.corrcoef(galton['childHeight'], galton['midparentHeight'])
print(cc)
cc[0,1]**2

## Multiple linear regression

In [ ]:
model2 = sm.OLS.from_formula('childHeight ~ father + mother + gender', data=galton)
result2 = model2.fit()
result2.summary()

- interpret the coefficients:
- add interaction terms:
- make a prediction on your data:

In [ ]:
# making prediction on new data
# input your father's height, mother's height and your gender
su_data = {'father':66, 'mother':64.5, 'gender':'female'}
# if you have multiple new data to predict, put them in a dataframe!
result2.predict(su_data)
# pretty good estimate on my height!

In [ ]:
# how to get CI and PI for new data: use get_prediction instead of the simple predict function
su_pred = result2.get_prediction(su_data)
# summary_frame will output SE, CI and PI
su_pred.summary_frame(alpha=0.05)
# CI is mean_ci, PI is obs_ci

### Real data: predict housing prices
A classic example in linear regression is to predict housing prices. The following dataset of house prices and predictors was collected over several years for the city of Ames, Iowa. A [description of the dataset appears online](http://ww2.amstat.org/publications/jse/v19n3/decock.pdf). We will overview modeling structure and carrying out exploratory data analysis. To begin, let's import our dataset.

In [ ]:
all_sales = pd.read_csv('house.csv')
all_sales

#### Investigate Dataset

Now that we have loaded our dataset, let's get a feel for what data our data looks like:

In [ ]:
all_sales.shape

Based on the above output, we have 2930 observations and 82 columns.  To get a better sense of the data, you can print all the column names. We will focus only a subset of the columns, and try to predict the sale price column from these columns. 

In [ ]:
all_sales.columns

In [ ]:
# only include single family homes and normal sale condition
sales = all_sales[(all_sales['Bldg Type']=='1Fam')&(all_sales['Sale Condition']=='Normal')]
# select the relevant columns 
sales = sales[['1st Flr SF', '2nd Flr SF', 
    'Total Bsmt SF', 'Garage Area', 
    'Wood Deck SF', 'Open Porch SF', 'Lot Area', 
    'Year Built', 'Yr Sold', 'SalePrice']]
sales.rename(columns={'1st Flr SF': 'FirstFlrSF', '2nd Flr SF': 'SecondFlrSF', 
    'Total Bsmt SF':'BasementSF', 'Garage Area':'GarageArea', 
    'Wood Deck SF':'DeckSF', 'Open Porch SF':'PorchSF', 'Lot Area':'LotArea', 
    'Year Built':'YrBuilt', 'Yr Sold':'YrSold'}, inplace=True)
sales = sales.reset_index(drop=True)
sales.head()

#### Handle Missing Data

Before we get started and discuss how to approach a modeling problem, it's good practice to observe our data thoroughly to identify missing values and handle them accordingly:

In [ ]:
sales.isnull().sum()

Fortunately, our output indicates that none of our columns contain missing values so we are able to continue on.

#### Modeling Structure

we will build a model to determine the estimated price of a home, our __dependent variable__ in the case is __SalePrice__. Our independent variables or predictors can be chosen from this datset, and based on their perceived relationship to our primary variable on interest, or price of home.

#### Exploratory Data Analysis

In [ ]:
sns.distplot(sales['SalePrice'], bins=30);

As you can see, our target variable follows roughly a normal distribution. However, there are outliers near the upper quantiles.

One of the most important steps in modeling is understanding the relationship our variables have with each other.  A relationship between two variables is also denoted as correlation is statistics.  A figure known as a correlation matrix, can be used to measure the linear relationships between variables.

In [ ]:
correlation_matrix = sales.corr().round(2)
correlation_matrix
sns.heatmap(data=correlation_matrix, annot=True);

Correlation coefficients range from -1 to 1, which determines the strength of correlation.  Values close to 1 signify a strong positive relationship, whereas values close to -1 indicate a strong negative relationship.

With this heatmap, we can see corellation coefficients for each of our potential covariates values and the target variable. 

To further investigate individual covariates, we can plot their values against the target variable. Seaborn pairplot can plot pair-wise scatter plot of all the variables in a dataframe. Let's pick a few of them and create a scatter plot matrix:

In [ ]:
sales_plot = sales[['FirstFlrSF', 'GarageArea', 'YrBuilt','SalePrice']]
sns.pairplot(sales_plot, diag_kind='hist');

In [ ]:
# this will take a while and the plots are too small to see
sns.pairplot(sales, diag_kind='hist');

#### Model with two variables

Our gut instinct tells us that the square footage would greatly impact the price of a home.  Additionally, we may want to consider the year built as a proxy for the condition of the house.

In [ ]:
bivar_model = sm.OLS.from_formula('SalePrice ~ FirstFlrSF + YrBuilt', data=sales)
result = bivar_model.fit()
result.summary()

Next, let's add another variable in the model. Now we have three predictors.

In [ ]:
trivar_model = sm.OLS.from_formula('SalePrice ~ FirstFlrSF + GarageArea + YrBuilt', data=sales)
result = trivar_model.fit()
result.summary()

We noted above that the regression coefficients for __FirstFlrSF__ and __YrBuilt__ both changed. In general, the estimated coefficient of a variable in a
regression model will change when other variables are added or
removed. A coefficient will be nearly unchanged if any variables that are added to or removed from the model are approximately **uncorrelated with the other covariates that are already in the model**.

Next we fit the full model by including all the other variables in the dataset. Notice if there are any categorical variables coded as numbers, for the model to treat it correctly, we need to include it in the formula as a categorical variable by using the syntax `C(VarName)`. This is in general the way to include categorical variables in the model: this will create K-1 dummy variables for a categorical variable with K categories, and treat one of the categories as baseline in the intercept. 

In [ ]:
full_model = sm.OLS.from_formula('SalePrice ~ FirstFlrSF + SecondFlrSF + BasementSF + GarageArea + DeckSF + PorchSF + LotArea + YrBuilt + YrSold', data=sales)
result = full_model.fit()
result.summary()

Let's now include only the significant predictors from the full model, and call this our final model. 

In [ ]:
final_model = sm.OLS.from_formula('SalePrice ~ FirstFlrSF + SecondFlrSF + BasementSF + GarageArea + DeckSF + LotArea + YrBuilt', data=sales)
result = final_model.fit()
result.summary()

## Visualization of the Fitted Models

Here we demonstrate some graphing techniques that can be
used to gain a better understanding of a regression model that has
been fit to data.

In [ ]:
sales_mean = round(sales.mean())
sales_mean

In [ ]:
from statsmodels.sandbox.predict_functional import predict_functional
? predict_functional

In [ ]:
# Fix certain variables at reference values
# Here we use average value in the dataset as reference
# but you can specify particular values

values = {'SecondFlrSF':sales_mean[1], 'BasementSF':sales_mean[2], 
          'GarageArea':sales_mean[3], 'DeckSF':sales_mean[4],
          'LotArea':sales_mean[6], 'YrBuilt':sales_mean[7]}
pred, CI, X = predict_functional(result, 'FirstFlrSF', values=values)

ax = sns.lineplot(X, pred, lw=5)
ax.fill_between(X, CI[:, 0], CI[:, 1], color='grey', alpha=0.4)
ax.set_xlabel('First Floor SF')
_ = ax.set_ylabel('House Sale Price')

Notice the change of the confidence band width over the range. Why is that?

Next is the analogous plot for YrBuilt is shown next. Notice the vertical axis is on a different scale! The confidence band is pretty even over the years. Does that make sense?

In [ ]:
values = {'FirstFlrSF':sales_mean[0], 'SecondFlrSF':sales_mean[1], 
          'BasementSF':sales_mean[2], 'GarageArea':sales_mean[3], 
          'DeckSF':sales_mean[4], 'LotArea':sales_mean[6]}
pred, CI, X = predict_functional(result, 'YrBuilt', values=values)

ax = sns.lineplot(X, pred, lw=5)
ax.fill_between(X, CI[:, 0], CI[:, 1], color='grey', alpha=0.4)
ax.set_xlabel('Year Built')
_ = ax.set_ylabel('House Sale Price')

Below we show the plot of residuals on fitted values.  It appears that we have a modestly increasing mean/variance relationship.  That is, the scatter around the mean house sales price is greater when the mean sale price itself is greater.

In [ ]:
pp = sns.scatterplot(result.fittedvalues, result.resid)
pp.set_xlabel('Fitted values')
_ = pp.set_ylabel('Residuals')

Below we show the plot of sales price on fitted values (predicted sales price). In a perfect model, this should scatter around the 45 degree line.

In [ ]:
pp = sns.scatterplot(result.fittedvalues, sales['SalePrice'])
pp.set_xlabel('Predicted SalePrice')
_ = pp.set_ylabel('SalePrice')

A "component plus residual plot" or "partial residual plot" is
intended to show how the data would look if all but one covariate
could be fixed at reference values.  By controlling the values of
these covariates, all remaining variation is due either to the "focus
variable" (the one variable that is left unfixed, and is plotted on
the horizontal axis), or to sources of variation that are unexplained
by any of our covariates.

For example, the partial residual plot below shows how FirstFlrSF (horizontal axis) and SalePrice (vertical axis) would be related if all other variables were fixed.  Note that the origin of the vertical axis in these plots is not meaningful (we are not implying that SalePrice would be negative), but the differences along the vertical axis are meaningful.  This plot implies that when all else are held fixed, the average SalePrice of two houses with 1000SF and 3000SF of first floor differ by
around 100K. This plot also shows, that the deviations from the
mean are somewhat smaller at the low end of the range compared to the
high end of the range, which makes sense.  We also see that at the high end of the range, the deviations from the mean are somewhat right-skewed, with exceptionally high SalePrice being more common than exceptionally low SalePrice.

In [ ]:
from statsmodels.graphics.regressionplots import plot_ccpr

ax = plt.axes()
plot_ccpr(result, 'FirstFlrSF', ax)
_ = ax.lines[0].set_alpha(0.2) # Reduce overplotting with transparency